# Prototypical UI Strategy 2

## 0. Prerequesites - Installing of packages and Loading Files

In [ ]:
import shap
import pickle
import pandas as pd
import numpy as np
import toml
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
import os
os.chdir("..")
config = toml.load("config.toml")
from src.utils import to_prob_list
from src.utils import *

from shap import Explanation

import toml
import sys

from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from src.preparation_pipeline import PreparationPipeline
from src.explain_pipeline import ExplainPipeline
from src.explainability_pipeline import ExplainabilityPipeline

from src.utils import plot_confusion_matrix


import matplotlib.pyplot as plt

import ipywidgets as widgets
from ipywidgets import AppLayout
from ipywidgets import HTML, Layout, Dropdown, Output, Textarea, VBox, Label
from ipywidgets import AppLayout, Button
from IPython.display import display, clear_output
from IPython.display import display

import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import Layout, Button, Box
%matplotlib inline

In [ ]:
base_path = config["result"]["results"]
strategy = "strategy 2/"
strengths = ["impure_0.0", "impure_0.1", "impure_0.2", "impure_0.3","impure_0.4","impure_0.5","impure_0.6","impure_0.7","impure_0.8","impure_0.9","impure_1.0"]
strength = 3

In [ ]:
with open(base_path + strategy + "2_xgb_shap_values_" + strengths[strength] + ".pckl", 'rb') as f:
     shap_values_xgb = pickle.load(f)
        
with open(base_path + "strategy 1/1_xgb_shap_values_" + strengths[strength] + ".pckl", 'rb') as f:
     shap_values_xgb_one = pickle.load(f)
        
with open(base_path + strategy + "2_xgb_class_proba_" + strengths[strength] + ".pckl", 'rb') as f:
     class_proba_xgb = pickle.load(f)
        
with open(base_path + strategy + "2_xgb_feature_names_" + strengths[strength] + ".pckl", 'rb') as f:
     feature_names = pickle.load(f)
        
with open(base_path + "strategy 1/1_xgb_y_test_" + strengths[strength] + ".pckl", 'rb') as f:
     y_test = pickle.load(f)

xgb_class = np.argmax(class_proba_xgb, axis=1) 
confusion_matrix_xgb = confusion_matrix(y_test, xgb_class)

## 1. UI Widget Definition

In [ ]:
def get_text_shap_feature_importance(shap_values_xgb, non_text = False):

    features = shap_values_xgb.data
    feature_names = shap_values_xgb.feature_names
    features = np.squeeze(np.asarray(shap_values_xgb.data))
    values = shap_values_xgb.values
    base_values = shap_values_xgb.base_values
    if non_text==True:
        feature_names = feature_names[:164] 
    else:
        feature_names = feature_names[164:]
    
    if non_text==True:
        features = features[:,0:164]
        #features = np.delete(features,np.s_[164:],axis=1)
        values = np.delete(values,np.s_[164:],axis=1)
    else: 
        features = features[:,164:]
        #features = np.delete(features,np.s_[0:164],axis=1)
        values = np.delete(values,np.s_[0:164],axis=1)
    expl = Explanation(values=values, base_values = base_values, feature_names = feature_names, data=features)
    return expl

In [ ]:
plt.rcParams["figure.dpi"] = 300
non_text_output_l = widgets.Output()
non_text_output_g1 = widgets.Output()
non_text_output_g2 = widgets.Output()
text_output_l1 = widgets.Output()
text_output_l2 = widgets.Output()
text_output_g1 = widgets.Output()
text_output_g2 = widgets.Output()
non_text_output_b = widgets.Output()
output_model_metrics_non_text = widgets.Output(width="30%")
output_model_metrics_text = widgets.Output()
output_model_metrics_result = widgets.Output()
output_model_num_metrics_result = widgets.Output()
pred_output = widgets.Output()
true_output = widgets.Output()

w = widgets.IntSlider(value=15,
    min=0,
    max=400,
    step=1,
    description='Case:',
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d')

box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    width='100%', height='50%')

box_layout_100 = Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    width='100%', height='100%')

box_layout_50 = Layout(display='flex',
                    flex_flow='column',
                    align_items='center',
                    width='50%', height='100%')

with pred_output:
    clear_output()
    out = "Rejected"
    if xgb_class[w.value] == 1:
        out = "Accepted"
    print("Predicted Label: " + out)
    
with true_output:
    clear_output()
    out = "Rejected"
    if y_test[w.value] == 1.0:
        out = "Accepted"
    print("True Label: " + out)

with text_output_l1:
    clear_output()
    shap.plots.waterfall(get_text_shap_feature_importance(shap_values_xgb, non_text = False)[w.value])
    
with text_output_l2:
    clear_output()
    #shap.plots.text(get_text_shap_feature_importance(shap_values_xgb, non_text = False)[w.value], cmax=0.3)    
    
with non_text_output_l:
    clear_output()
    shap.plots.waterfall(get_text_shap_feature_importance(shap_values_xgb, non_text = True)[w.value])
    
with non_text_output_b:
    clear_output()
    shap.plots.bar(shap_values_xgb, max_display=12, show=False)
    plt.show()
with non_text_output_g1:
    clear_output()
    shap.plots.bar(get_text_shap_feature_importance(shap_values_xgb, non_text = True), show=False)
    plt.show()
with non_text_output_g2:
    clear_output()
    shap.plots.beeswarm(get_text_shap_feature_importance(shap_values_xgb, non_text = True),show=False)
    plt.show()
with text_output_g1:
    clear_output()
    shap.plots.bar(get_text_shap_feature_importance(shap_values_xgb, non_text = False), show=False)
    plt.show()
with text_output_g2:
    clear_output()
    plt = n_important_words_wordcloud(get_text_shap_feature_importance(shap_values_xgb, non_text = False),100)
    plt.show()

with output_model_metrics_result:
    clear_output()
    plt.show(plot_confusion_matrix(confusion_matrix_xgb))
    
with output_model_num_metrics_result:
    clear_output()
    #print(classification_report(y_test, xgb_class))
    print("ROC AUC Score: " + str(round(roc_auc_score(y_test, xgb_class),2)))
    print("F1-Score:      " + str(round(f1_score(y_test, xgb_class),2)))
    print("Accuracy:      " + str(round(accuracy_score(y_test, xgb_class),2)))
    
def on_value_change(change):
    with text_output_l1:
        clear_output()
    with text_output_l2:
        clear_output()
    with non_text_output_l:
        clear_output()
    with text_output_l1:
        shap.plots.waterfall(get_text_shap_feature_importance(shap_values_xgb, non_text = False)[w.value])
    with non_text_output_l:
        shap.plots.waterfall(get_text_shap_feature_importance(shap_values_xgb, non_text = True)[w.value])
    with pred_output:
        clear_output()
        out = "Rejected"
        if xgb_class[w.value] == 1:
            out = "Accepted"
        print("Predicted Label: " + out)
    with true_output:
        clear_output()
        out = "Rejected"
        if y_test[w.value] == 1.0:
            out = "Accepted"
        print("True Label: " + out)
    
w.observe(on_value_change, names='value')

In [ ]:
from ipywidgets import Button, GridBox, Layout, ButtonStyle

box0 = Box(children=[non_text_output_b], layout=box_layout_50)
box1 = Box(children=[text_output_g1], layout=box_layout)
box2 = Box(children=[text_output_g2], layout=box_layout)
box3 = Box(children=[true_output], layout=box_layout_100)
box4 = Box(children=[pred_output], layout=box_layout_100)

shap_bar_plot  = Button(description='SHAP Bar Plot',
                 layout=Layout(width='100%'),
                 style=ButtonStyle(button_color='WhiteSmoke', text_color="White", font_weight="bold"))

true_label  = Button(description="True Label",
                 layout=Layout(width='100%'),
                 style=ButtonStyle(button_color='WhiteSmoke', text_color="White", font_weight="bold"))

prediction  = Button(description="Predicted Label",
                 layout=Layout(width='100%'),
                 style=ButtonStyle(button_color='WhiteSmoke', text_color="White", font_weight="bold"))

shap_force_plot  = Button(description='SHAP Force Plot',
                 layout=Layout(width='100%'),
                 style=ButtonStyle(button_color='WhiteSmoke', text_color="White", font_weight="bold"))

shap_beeswarm_plot  = Button(description='SHAP Beeswarm Plot',
                 layout=Layout(width='100%'),
                 style=ButtonStyle(button_color='WhiteSmoke', text_color="white", font_weight="bold"))

wordcloud = Button(description='Wordcloud 100 Most Important Words',
                 layout=Layout(width='100%'),
                 style=ButtonStyle(button_color='WhiteSmoke', text_color="white", font_weight="bold"))

shap_text_plot = Button(description='SHAP Text Plot',
                 layout=Layout(width='100%'),
                 style=ButtonStyle(button_color='WhiteSmoke', text_color="white", font_weight="bold"))

shap_waterfall_plot = Button(description='SHAP Waterfall Plot',
                 layout=Layout(width='100%'),
                 style=ButtonStyle(button_color='WhiteSmoke', text_color="white", font_weight="bold"))

pred = widgets.Accordion([VBox(children=[ true_label,
                                     box3, prediction, box4])],layout=Layout(width='100%'))
pred.set_title(0, 'Resulting Model Prediction')


text_g = widgets.Accordion([VBox(children=[ shap_bar_plot,
                                     box1,wordcloud, box2])],layout=Layout(width='50%'))
text_g.set_title(0, 'Textual Features')

box3 = Box(children=[non_text_output_l], layout=box_layout_100)


non_text_l = widgets.Accordion([VBox(children=[ shap_waterfall_plot,
                                     box3])], layout=Layout(width='50%'))

non_text_l.set_title(0, 'Non-Textual Features')

box4 = Box(children=[text_output_l1], layout=box_layout_100)
box5 = Box(children=[text_output_l2], layout=box_layout)

text_l = widgets.Accordion([VBox(children=[ shap_waterfall_plot,
                                     box4])], layout=Layout(width='50%'))
text_l.set_title(0, 'Textual Features')

box6 = Box(children=[non_text_output_g1], layout=box_layout)
box7 = Box(children=[non_text_output_g2], layout=box_layout)

both_g = widgets.Accordion([VBox(children=[ shap_bar_plot, box0])], layout=Layout(width='99%'))
both_g.set_title(0, 'Combined Features')


non_text_g = widgets.Accordion([VBox(children=[ shap_bar_plot,
                                     box6, shap_beeswarm_plot, box7])], layout=Layout(width='50%'))
non_text_g.set_title(0, 'Non-Textual Features')


metrics_result = widgets.Accordion([output_model_metrics_result], layout=Layout(width='50%'))
metrics_result.set_title(0, 'Resulting Model Confusion Matrix')

num_metrics_result = widgets.Accordion([output_model_num_metrics_result], layout=Layout(width='50%', height='100%'))
num_metrics_result.set_title(0, 'Resulting Model Metrics')

tab1 = VBox(children=[both_g, HBox(children=[
                                    non_text_g, text_g]),
                      ])
#true_label,true_output, prediction, pred_output
                                     
tab2 = VBox(children=[VBox(children=[w,pred , HBox(children=[
                                     non_text_l, text_l]),
                             ])
                      ])

tab3 = VBox(children=[HBox(children=[ 
                                     metrics_result, num_metrics_result]),
                                     ])
tab = widgets.Tab(children=[tab1, tab2, tab3])
tab.set_title(0, 'Global Explainability')
tab.set_title(1, 'Local Explainability')
tab.set_title(2, 'Model Metrics')

## 2. UI Widget Presentation

In [ ]:
VBox(children=[tab])